# Fragrance Segmentation – Unsupervised Clustering Analysis

_Course: Machine Learning_

- Name: Muhammad Umar Uz Zaman  
- Student ID: STU1197819  
- Goal: I explore a large fragrance dataset to understand:
  - How perfumes naturally group together based on their scent notes and key characteristics
  - Which fragrance segments tend to be more affordable versus more premium or luxury
  - How these segments can support targeted marketing, product positioning, and personalised recommendations
- Dataset: `Fragrance Dataset 4037.csv` (records of perfumes with information on notes and other descriptive attributes)

**Phases of this notebook**

Intro & setup  
Phase 1 – Data loading & structure check  
Phase 2 – Data cleaning, preparation & feature selection  
Phase 3 – Core EDA (distributions, relationships & feature importance)  
Phase 4 – Clustering models (K-Means, Hierarchical, DBSCAN) & evaluation  
Phase 5 – Cluster profiling & business insights  
Phase 6 – Wrap-up, limitations & export for management report


### Phase 1

In [9]:
# Core libraries
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Clustering and preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Hierarchical clustering tools for dendrograms
from scipy.cluster.hierarchy import dendrogram, linkage

# Plotting style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.titlesize"] = 12
plt.rcParams["axes.labelsize"] = 11
plt.rcParams["figure.dpi"] = 100


In [13]:
# Path to the fragrance dataset
data_path = "Fragrance Dataset.csv"  # Use the CSV file instead of Excel

# Load the dataset
df = pd.read_csv(data_path)

print("Shape of the dataset (rows, columns):", df.shape)
print("\nColumn names:")
print(df.columns.tolist())

# First few rows to see how the raw data looks
df.head()


Shape of the dataset (rows, columns): (1000, 10)

Column names:
['brand', 'title', 'type', 'price', 'priceWithCurrency', 'available', 'availableText', 'sold', 'lastUpdated', 'itemLocation']


,brand,title,type,price,priceWithCurrency,available,availableText,sold,lastUpdated,itemLocation
0,Dior,Christian Dior Sauvage Men's EDP 3.4 oz Fragra...,Eau de Parfum,84.99,US $84.99/ea,10.0,More than 10 available / 116 sold,116.0,"May 24, 2024 10:03:04 PDT","Allen Park, Michigan, United States"
1,AS SHOW,A-v-entus Eau de Parfum 3.3 oz 100ML Millesime...,Eau de Parfum,109.99,US $109.99,8.0,8 available / 48 sold,48.0,"May 23, 2024 23:07:49 PDT","Atlanta, Georgia, Canada"
2,Unbranded,HOGO BOSS cologne For Men 3.4 oz,Eau de Toilette,100.00,US $100.00,10.0,More than 10 available / 27 sold,27.0,"May 22, 2024 21:55:43 PDT","Dearborn, Michigan, United States"
3,Giorgio Armani,Acqua Di Gio by Giorgio Armani 6.7 Fl oz Eau D...,Eau de Toilette,NaN,US $44.99/ea,2.0,2 available / 159 sold,159.0,"May 24, 2024 03:30:43 PDT","Reinholds, Pennsylvania, United States"
4,Lattafa,Lattafa Men's Hayaati Al Maleky EDP Spray 3.4 ...,Fragrances,16.91,US $16.91,NaN,Limited quantity available / 156 sold,156.0,"May 24, 2024 07:56:25 PDT","Brooklyn, New York, United States"


In [14]:
# Overall information about the dataframe
df.info()

# Missing values per column
print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False))

# Summary statistics for numeric columns
numeric_summary = df.describe(include=[np.number]).T
numeric_summary


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              999 non-null    object 
 1   title              1000 non-null   object 
 2   type               997 non-null    object 
 3   price              996 non-null    float64
 4   priceWithCurrency  1000 non-null   object 
 5   available          889 non-null    float64
 6   availableText      997 non-null    object 
 7   sold               994 non-null    float64
 8   lastUpdated        947 non-null    object 
 9   itemLocation       1000 non-null   object 
dtypes: float64(3), object(7)
memory usage: 78.2+ KB

Missing values per column:
available            111
lastUpdated           53
sold                   6
price                  4
availableText          3
type                   3
brand                  1
title                  0
priceWithCurrency      0
itemLocation

,count,mean,std,min,25%,50%,75%,max
price,996.0,46.497279,35.590865,3.0,22.99,35.71,59.0,259.09
available,889.0,20.046119,61.547985,2.0,5.00,10.00,10.0,842.00
sold,994.0,766.266600,3200.971733,1.0,14.00,49.50,320.5,54052.00


From this structural check I can see that the dataset has a mix of numeric and categorical information:

- There are three numeric columns: `price`, `available` and `sold`. These capture the commercial side of each perfume listing: how much it costs, how many units are available and how many units have been sold.
- The remaining columns are stored as text, including `brand`, `title`, `type`, `priceWithCurrency`, `availableText`, `lastUpdated` and `itemLocation`. These fields describe the product, the listing text, the time of the last update and where the item is located.

In terms of data quality:

- The `price` and `sold` columns are almost complete, with only a handful of missing values, so I expect to rely on them strongly in my analysis.
- The `available` column has more missing entries, which suggests that availability information is not always reported or not always parsed correctly.
- The `lastUpdated` and `availableText` fields are mostly complete but are stored as free text, which means they will be harder to use directly in a clustering algorithm.
- The basic statistics show that `price`, `available` and especially `sold` are all quite skewed, with a few very large values. This is typical for online marketplace data, where a small number of listings can have very high stock or sales.

Overall, this first check reassures me that I have several usable numeric features for clustering, while also highlighting that I need to be careful with skewed distributions and with how I handle missing values in `available`. Before I move on to cleaning and feature selection, I briefly look at the structure of the most important categorical columns.


In [15]:
# Quick look at the structure of key categorical columns
cat_cols = ["brand", "type", "itemLocation"]

for col in cat_cols:
    print(f"\nColumn: {col}")
    print(f"Number of unique values: {df[col].nunique()}")
    print("Most frequent values:")
    print(df[col].value_counts().head(10))



Column: brand
Number of unique values: 246
Most frequent values:
brand
Giorgio Armani        60
Paco Rabanne          45
Ralph Lauren          41
Unbranded             39
Armaf                 34
Dolce&Gabbana         32
Yves Saint Laurent    30
Versace               20
Calvin Klein          17
Cologne               16
Name: count, dtype: int64

Column: type
Number of unique values: 63
Most frequent values:
type
Eau de Toilette    444
Eau de Parfum      285
Eau de Cologne      56
Perfume             50
Fragrances          32
Eau De Parfum       15
Parfum              13
Eau de Perfume      10
Eau De Toilette      9
Gift Sets            6
Name: count, dtype: int64

Column: itemLocation
Number of unique values: 276
Most frequent values:
itemLocation
Dallas, Texas, United States             126
Hackensack, New Jersey, United States    105
Brooklyn, New York, United States         76
Miami, Florida, United States             41
Edison, New Jersey, United States         35
Houston, Texas, 

The categorical overview gives me a clearer picture of the breadth of the dataset:

- The `brand` column has 246 unique values, with a long tail beyond the most common brands such as Giorgio Armani, Paco Rabanne and Ralph Lauren. This tells me that brand is highly granular and better suited for describing clusters later rather than for direct use as a modelling feature.
- The `type` column has 63 unique values, but the majority of listings fall into a few familiar fragrance formats such as Eau de Toilette, Eau de Parfum, Eau de Cologne and Perfume. This suggests that I can simplify `type` into a smaller set of meaningful groups without losing much information.
- The `itemLocation` field contains many different city and state combinations. This provides useful commercial context (for example, where sellers are based), but the variety of locations means that it is unlikely to be a stable driver of fragrance similarity. I plan to use it mainly for interpretation rather than as a clustering feature.

These observations already point towards a feature-selection strategy where I focus on the core numeric variables (`price`, `available`, `sold`) and a simplified version of `type` for clustering, while keeping `brand` and `itemLocation` for later profiling of the resulting segments.
